<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/transcripci%C3%B3n_autom%C3%A1tica/transcripci%C3%B3n_de_videos_de_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reconocimiento automático de voz

La tarea de reconocimiento de voz, también conocida como transcripción, es capaz de convertir el lenguaje hablado (una señal de audio) en texto escrito. Conocido por sus siglas en ingles, el ASR (*Automatic Speech Recognition*) se utiliza comúnmente en aplicaciones orientadas al usuario, como agentes virtuales, generación de subtítulos en vivo, toma de notas, etc. La transcripción precisa del habla es esencial para estos casos de uso.

ASR es un componente crítico de la inteligencia artificial. Se usan también terminologías alternativas para describir el reconocimiento de voz como conversión de voz a texto o STT (*speech-to-text*).

### Whisper (Open AI)

Whisper es un sistema de reconocimiento automático de voz (ASR) entrenado en 680,000 horas de datos supervisados multilingües y multitarea recopilados de la web. Mostramos que el uso de un conjunto de datos tan grande y diverso conduce a una mayor solidez a los acentos, el ruido de fondo y el lenguaje técnico. Además, permite la transcripción en varios idiomas, así como la traducción de esos idiomas al inglés. Somos modelos de código abierto y código de inferencia que sirven como base para crear aplicaciones útiles y para futuras investigaciones sobre procesamiento de voz sólido.

In [ ]:
!pip install gradio

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

In [ ]:
!pip install pytube

In [ ]:
!nvidia-smi

Sat Feb 18 00:45:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import whisper
import gradio as gr
from pytube import YouTube

In [ ]:
class GradioInference():
  def __init__(self):
    self.sizes = list(whisper._MODELS.keys())
    self.langs = ["none"] + sorted(list(whisper.tokenizer.LANGUAGES.values()))
    self.current_size = "base"
    self.loaded_model = whisper.load_model(self.current_size)
    self.yt = None
  
  def __call__(self, link, lang, size, subs):
    if self.yt is None:
      self.yt = YouTube(link)
    path = self.yt.streams.filter(only_audio=True)[0].download(filename="tmp.mp4")

    if lang == "none":
      lang = None

    if size != self.current_size:
      self.loaded_model = whisper.load_model(size)
      self.current_size = size
    results = self.loaded_model.transcribe(path, language=lang)

    if subs == "None":
      return results["text"]
    elif subs == ".srt":
      return self.srt(results["segments"])
    elif ".csv" == ".csv":
      return self.csv(results["segments"])
   
  def srt(self, segments):
    output = ""
    for i, segment in enumerate(segments):
      output += f"{i+1}\n"
      output += f"{self.format_time(segment['start'])} --> {self.format_time(segment['end'])}\n"
      output += f"{segment['text']}\n\n"
    return output
  
  def csv(self, segments):
    output = ""
    for segment in segments:
      output += f"{segment['start']},{segment['end']},{segment['text']}\n"
    return output

  def format_time(self, time):
    hours = time//3600
    minutes = (time - hours*3600)//60
    seconds = time - hours*3600 - minutes*60
    milliseconds = (time - int(time))*1000
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"
    
  def populate_metadata(self, link):
    self.yt = YouTube(link)
    return self.yt.thumbnail_url, self.yt.title

gio = GradioInference()
title="Transcripción de videos de YouTube"
description="Transcripción de voz a texto de videos de Youtube utilizando Whisper de OpenAI"

block = gr.Blocks()
with block:
    gr.HTML(
        """
            <div style="text-align: center; max-width: 500px; margin: 0 auto;">
              <div>
                <h1>Transcripción de videos de YouTube</h1>
              </div>
              <p style="margin-bottom: 10px; font-size: 94%">
                Transcripción de voz a texto de videos de Youtube utilizando Whisper de OpenAI
              </p>
            </div>
        """
    )
    with gr.Group():
        with gr.Box():
          with gr.Row().style(equal_height=True):
            sz = gr.Dropdown(label="Tamaño del Modelo", choices=gio.sizes, value='base')
            lang = gr.Dropdown(label="Idioma (Opcional)", choices=gio.langs, value="none")
          with gr.Row().style(equal_height=True):
            wt = gr.Radio(["No", ".srt", ".csv"], label="¿Con marcas de tiempo?")
          link = gr.Textbox(label="Enlace a YouTube")
          title = gr.Label(label="Título del Video")
          with gr.Row().style(equal_height=True):
            img = gr.Image(label="Miniatura")
            text = gr.Textbox(label="Transcripción", placeholder="Salida de la Transcripción", lines=10)
          with gr.Row().style(equal_height=True): 
              btn = gr.Button("Transcribir")
          btn.click(gio, inputs=[link, lang, sz, wt], outputs=[text])
          link.change(gio.populate_metadata, inputs=[link], outputs=[img, title])
block.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a8be6c34-8609-4845.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


**Referencias**

* https://gradio.app/
* https://openai.com/blog/whisper/
* https://dev.to/zirkelc/automatically-transcribe-youtube-videos-with-openai-whisper-1856